In [31]:
pip install selenium==3.141.0

  Attempting uninstall: selenium
    Found existing installation: selenium 4.6.0
    Uninstalling selenium-4.6.0:
      Successfully uninstalled selenium-4.6.0
Note: you may need to restart the kernel to use updated packages.


In [119]:
import selenium
from selenium import webdriver
import re
from bs4 import BeautifulSoup

In [2]:
site = 'https://veiculos.fipe.org.br/' 


In [3]:
driver = webdriver.Chrome('C:\\Users\\prpau\\Documents\\chromedriver.exe')
driver.get(site)

In [150]:
# consulta de carros e utilitários, copia o xpath abaixo
xpath_pesquisa = '//*[@id="front"]/div[1]/div[1]/ul/li[1]/a/div[2]'

id_marca = 'selectMarcacarro_chosen'
id_modelo = 'selectAnoModelocarro_chosen'
id_ano = 'selectAnocarro_chosen'
id_tabela = 'selectTabelaReferenciacarro_chosen'

id_botao_limpar = 'buttonLimparPesquisarcarro'
id_botao_pesq = 'buttonPesquisarCarro'

id_resultado = 'resultadoConsultacarroFiltros'

ano_veiculo = '2013'
modelo_veiculo = 'Gol (novo) 1.0 Mi Total Flex 8V 4p'
marca_modelo = "VW - Volkswagen"



In [151]:
def xpath_input (id_elemento):
    return '//*[@id="{}"]/div/div/input'.format(id_elemento)

In [93]:
import random
import time

def esperar(segundos = None):
    
    if not segundos:
        segundos = random.randrange(3,8)
        
    time.sleep(segundos)

In [107]:
driver = webdriver.Chrome('C:\\Users\\prpau\\Documents\\chromedriver.exe')
driver.get(site)
esperar(10)
tipo_consulta = driver.find_element_by_xpath(xpath_pesquisa)
tipo_consulta.click()

In [86]:
pesquisar = driver.find_element_by_id(id_tabela)
pesquisar.click()



from selenium.webdriver.common.keys import Keys

xpath = xpath_input(id_tabela)

selecao = driver.find_element_by_xpath(xpath)


selecao.send_keys('fevereiro/2021' + Keys.DOWN + Keys.RETURN)

In [88]:
pesquisar.is_selecteds = False
selecao.is_seleecteds = False

In [108]:
def preenche_item(id_item, texto_pesquisa):
    
    pesquisar = driver.find_element_by_id(id_item)
    pesquisar.click()
    
    xpath = xpath_input(id_item)
    selecao = driver.find_element_by_xpath(xpath)
    
    selecao.send_keys(texto_pesquisa + Keys.DOWN + Keys.RETURN)
    
    pesquisar.is_selecteds = False
    selecao.is_seleecteds = False

In [152]:
esperar()
preenche_item(id_tabela, 'fevereiro/2021')

In [110]:
esperar()
preenche_item(id_marca, marca_modelo )

In [111]:
esperar()
preenche_item(id_modelo, modelo_veiculo)

In [112]:
esperar()
preenche_item(id_ano, ano_veiculo)

In [113]:
esperar()
botao_pesquisa = driver.find_element_by_xpath('//*[@id="front"]/div[1]/div[1]/ul/li[1]/div/article[1]/div[3]/div[3]')
botao_pesquisa.click()

In [126]:
tabela = driver.find_element_by_id(id_resultado)
html = tabela.get_attribute('innerHTML')
parser = BeautifulSoup(html,'html.parser')
tabela = parser.select_one('tbody')
tabela

<tbody>
<tr>
<td class="noborder"><p>Mês de referência:</p></td>
<td><p>fevereiro de 2021 </p></td>
</tr>
<tr>
<td class="noborder"><p>Código Fipe:</p></td>
<td><p>005275-2</p></td>
</tr>
<tr>
<td class="noborder"><p>Marca:</p></td>
<td><p>VW - VolksWagen</p></td>
</tr>
<tr>
<td class="noborder"><p>Modelo:</p></td>
<td><p>Gol (novo) 1.0 Mi Total Flex 8V 4p</p></td>
</tr>
<tr>
<td class="noborder"><p>Ano Modelo:</p></td>
<td><p>2013 Gasolina</p></td>
</tr>
<tr>
<td class="noborder"><p>Autenticação</p></td>
<td><p>l7sn2y2ymxnc</p></td>
</tr>
<tr>
<td class="noborder"><p>Data da consulta</p></td>
<td><p>segunda-feira, 7 de novembro de 2022 14:29</p></td>
</tr>
<tr class="last">
<td class="noborder"><p>Preço Médio</p></td>
<td><p>R$ 23.579,00</p></td>
</tr>
</tbody>

In [146]:
regex_resultado = re.compile('(<td><p>)(.*?)(</p></td>)')
dados = regex_resultado.findall(str(tabela))

lista_dados=[]

import numpy as np

for i in dados:
        lista_dados.append(i[1])

np.array(lista_dados).reshape(1,-1)

array([['fevereiro de 2021 ', '005275-2', 'VW - VolksWagen',
        'Gol (novo) 1.0 Mi Total Flex 8V 4p', '2013 Gasolina',
        'l7sn2y2ymxnc', 'segunda-feira, 7 de novembro de 2022 14:29',
        'R$ 23.579,00']], dtype='<U42')

In [128]:
regex_titulo = re.compile('(<td class="noborder"><p>)(.*?)(</p></td>)')
dados_titulo = regex_titulo.findall(str(tabela))
dados_titulo

[('<td class="noborder"><p>', 'Mês de referência:', '</p></td>'),
 ('<td class="noborder"><p>', 'Código Fipe:', '</p></td>'),
 ('<td class="noborder"><p>', 'Marca:', '</p></td>'),
 ('<td class="noborder"><p>', 'Modelo:', '</p></td>'),
 ('<td class="noborder"><p>', 'Ano Modelo:', '</p></td>'),
 ('<td class="noborder"><p>', 'Autenticação', '</p></td>'),
 ('<td class="noborder"><p>', 'Data da consulta', '</p></td>'),
 ('<td class="noborder"><p>', 'Preço Médio', '</p></td>')]

In [147]:
import pandas as pd
titulos = []

for i in dados_titulo:
    
    titulos.append(i[1].replace(':',''))

df = pd.DataFrame( data = np.array(lista_dados).reshape(1,-1), columns = titulos)
df

,Mês de referência,Código Fipe,Marca,Modelo,Ano Modelo,Autenticação,Data da consulta,Preço Médio
0,fevereiro de 2021,005275-2,VW - VolksWagen,Gol (novo) 1.0 Mi Total Flex 8V 4p,2013 Gasolina,l7sn2y2ymxnc,"segunda-feira, 7 de novembro de 2022 14:29","R$ 23.579,00"


In [148]:
esperar()
botao_limpar = driver.find_element_by_id(id_botao_limpar)
botao_limpar.click()

In [168]:
#####



xpath = xpath_input(id_tabela)
selecao = driver.find_element_by_xpath(xpath)
pesquisar = driver.find_element_by_id(id_tabela)
pesquisar.click()
selecao.send_keys('fevereiro')
html = pesquisar.get_attribute('innerHTML')
re_anos = re.compile(r'(" style="">)(.*?)(</li>)')

lista = re_anos.findall(str(html))
lista



#########

[('" style="">', '<em>fevereiro</em>/2022 ', '</li>'),
 ('" style="">', '<em>fevereiro</em>/2021 ', '</li>'),
 ('" style="">', '<em>fevereiro</em>/2020 ', '</li>'),
 ('" style="">', '<em>fevereiro</em>/2019 ', '</li>'),
 ('" style="">', '<em>fevereiro</em>/2018 ', '</li>'),
 ('" style="">', '<em>fevereiro</em>/2017 ', '</li>'),
 ('" style="">', '<em>fevereiro</em>/2016 ', '</li>'),
 ('" style="">', '<em>fevereiro</em>/2015 ', '</li>'),
 ('" style="">', '<em>fevereiro</em>/2014 ', '</li>'),
 ('" style="">', '<em>fevereiro</em>/2013 ', '</li>'),
 ('" style="">', '<em>fevereiro</em>/2012 ', '</li>'),
 ('" style="">', '<em>fevereiro</em>/2011 ', '</li>'),
 ('" style="">', '<em>fevereiro</em>/2010 ', '</li>'),
 ('" style="">', '<em>fevereiro</em>/2009 ', '</li>'),
 ('" style="">', '<em>fevereiro</em>/2008 ', '</li>'),
 ('" style="">', '<em>fevereiro</em>/2007 ', '</li>'),
 ('" style="">', '<em>fevereiro</em>/2006 ', '</li>'),
 ('" style="">', '<em>fevereiro</em>/2005 ', '</li>'),
 ('" style

In [26]:
html = pesquisar.get_attribute('innerHTML')
re_anos = re.compile(r'(" style="">)(.*?)(</li>)')

lista = re_anos.findall(str(html))
lista

[('" style="">', 'novembro/2022 ', '</li>'),
 ('" style="">', 'outubro/2022 ', '</li>'),
 ('" style="">', 'setembro/2022 ', '</li>'),
 ('" style="">', 'agosto/2022 ', '</li>'),
 ('" style="">', 'julho/2022 ', '</li>'),
 ('" style="">', 'junho/2022 ', '</li>'),
 ('" style="">', 'maio/2022 ', '</li>'),
 ('" style="">', 'abril/2022 ', '</li>'),
 ('" style="">', 'março/2022 ', '</li>'),
 ('" style="">', 'fevereiro/2022 ', '</li>'),
 ('" style="">', 'janeiro/2022 ', '</li>'),
 ('" style="">', 'dezembro/2021 ', '</li>'),
 ('" style="">', 'novembro/2021 ', '</li>'),
 ('" style="">', 'outubro/2021 ', '</li>'),
 ('" style="">', 'setembro/2021 ', '</li>'),
 ('" style="">', 'agosto/2021 ', '</li>'),
 ('" style="">', 'julho/2021 ', '</li>'),
 ('" style="">', 'junho/2021 ', '</li>'),
 ('" style="">', 'maio/2021 ', '</li>'),
 ('" style="">', 'abril/2021 ', '</li>'),
 ('" style="">', 'março/2021 ', '</li>'),
 ('" style="">', 'fevereiro/2021 ', '</li>'),
 ('" style="">', 'janeiro/2021 ', '</li>'),
 ('

In [27]:
datas = []
for i in lista:
    datas.append(i[1])
    
datas

['novembro/2022 ',
 'outubro/2022 ',
 'setembro/2022 ',
 'agosto/2022 ',
 'julho/2022 ',
 'junho/2022 ',
 'maio/2022 ',
 'abril/2022 ',
 'março/2022 ',
 'fevereiro/2022 ',
 'janeiro/2022 ',
 'dezembro/2021 ',
 'novembro/2021 ',
 'outubro/2021 ',
 'setembro/2021 ',
 'agosto/2021 ',
 'julho/2021 ',
 'junho/2021 ',
 'maio/2021 ',
 'abril/2021 ',
 'março/2021 ',
 'fevereiro/2021 ',
 'janeiro/2021 ',
 'dezembro/2020 ',
 'novembro/2020 ',
 'outubro/2020 ',
 'setembro/2020 ',
 'agosto/2020 ',
 'julho/2020 ',
 'junho/2020 ',
 'maio/2020 ',
 'abril/2020 ',
 'março/2020 ',
 'fevereiro/2020 ',
 'janeiro/2020 ',
 'dezembro/2019 ',
 'novembro/2019 ',
 'outubro/2019 ',
 'setembro/2019 ',
 'agosto/2019 ',
 'julho/2019 ',
 'junho/2019 ',
 'maio/2019 ',
 'abril/2019 ',
 'março/2019 ',
 'fevereiro/2019 ',
 'janeiro/2019 ',
 'dezembro/2018 ',
 'novembro/2018 ',
 'outubro/2018 ',
 'setembro/2018 ',
 'agosto/2018 ',
 'julho/2018 ',
 'junho/2018 ',
 'maio/2018 ',
 'abril/2018 ',
 'março/2018 ',
 'fevereiro/

In [114]:




#from selenium.webdriver.common.action_chains import ActionChains


#driver.implicitly_wait(10)
#ActionChains(driver).move_to_element(selecao).click(selecao).perform()

selecao.send_keys('fevereiro/2022' + Keys.DOWN + Keys.RETURN)

ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=107.0.5304.88)


In [169]:
from scipy.stats import binom  

In [173]:
binom.pmf(1,2,1/6) + binom.pmf(2,2,1/6)

0.3055555555555555

In [1]:
import matplotlib.pyplot as plt

In [2]:
x = list(range(1,11))
x

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [ ]:
y = [359,1539,4353,8123,10000,8123,]